## Generate Customer Data

### Use some features from 'santander product recommendation' to generate customer data:
- Change the columns name
- Remove duplicated values of customer_id from the original dataset
- Update and mapping the values of the following features:
    - Age: Remove all samples having age >= 100 & age < 18 & age = 'NA'
    - First Join Date: Change from Object to Date

- Newly generated data:
     - Gender: 0 as Male and 1 as Female
    - Channel Entrance: Replace the missing value with synthetic branch code called KDB
    - Annual Household Gross Income: Generate new gross income values based on log-normal distribution. Mean and Median are based on UK statistics.
        - Mean = £47,400
        - Median = £34,200
    - Annual Personal Gross Income: Random proportions of 0.5 to 1 of household income
    
    - Pension: Randomly replace nan values with 1 or 0
    - Activity status: Re-generate the values with the same reason above

    - Employment status: 1 means Employed, 0 means Unemployed
    - Current_loan_amount:
        - Only people having 1 in Loans product have this value
        - Loans amounts correlate with income
    
    - Credit Score: Generated based on Experian Credit score range in the UK, ranging from 0-999. Score bands:
        - Very Poor: 0-560 
        - Poor: 561-720 
        - Fair/Average: 721-880 
        - Good: 881-960 
        - Excellent: 961-999
    - Customer Segment: 
        - There are three segments: 0-1 years, 2-4 years, >= 5 years
                

*Notes:
- The residence_country values is already in ISO 3166-1 country codes

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('original-datasets-from-kaggle/santander-product-recommendation/train_ver2.csv', low_memory=False)

In [3]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [ ]:
# Load the excel file having filters of features for keeping
filter_df = pd.read_excel('investigate.xlsx')
filter_df

,Column Name,Description,Medata Field,Keep,Note
0,ncodpers,Customer code,customer_id,1.0,duplicated values
1,pais_residencia,Customer's Country residence,residence_country,1.0,ISO format code
2,sexo,Customer's sex,gender,1.0,binary 1 or 0
3,age,Age,age,1.0,NaN
4,fecha_alta,The date in which the customer became as the f...,first_join_date,1.0,The date a customer first joined the bank
5,indresi,Residence index (S (Yes) or N (No) if the resi...,residence_index,1.0,Residence index (S (Yes) or N (No) if the resi...
6,canal_entrada,channel used by the customer to join,channel_entrace,1.0,channel used by the customer to join
7,ind_actividad_cliente,"Activity index (1, active customer; 0, inactiv...",activity_status,1.0,"Activity index (1, active customer; 0, inactiv..."
8,renta,Gross income of the household,household_gross_income,1.0,Gross income of the household
9,ind_ahor_fin_ult1,Saving Account,saving_account,1.0,Saving Account


In [5]:
temp = filter_df['Column Name'].tolist()
drop_features = [i for i in train_df.columns if i not in temp]
train_df.drop(drop_features, axis=1, inplace=True)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 16 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ncodpers               int64  
 1   pais_residencia        object 
 2   sexo                   object 
 3   age                    object 
 4   fecha_alta             object 
 5   indresi                object 
 6   canal_entrada          object 
 7   ind_actividad_cliente  float64
 8   renta                  float64
 9   ind_ahor_fin_ult1      int64  
 10  ind_aval_fin_ult1      int64  
 11  ind_ctju_fin_ult1      int64  
 12  ind_pres_fin_ult1      int64  
 13  ind_tjcr_fin_ult1      int64  
 14  ind_nom_pens_ult1      float64
 15  ind_recibo_ult1        int64  
dtypes: float64(3), int64(7), object(6)
memory usage: 1.6+ GB


In [7]:
# Rename the columns and drop duplicated values of customer_id
new_features_name = filter_df['Medata Field'].tolist()[:16]
train_df.rename(columns=dict(zip(train_df.columns, new_features_name)), inplace=True)
train_df.drop_duplicates('customer_id', keep='first', inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 956645 entries, 0 to 13647308
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             956645 non-null  int64  
 1   residence_country       949305 non-null  object 
 2   gender                  949300 non-null  object 
 3   age                     956645 non-null  object 
 4   first_join_date         949305 non-null  object 
 5   residence_index         949305 non-null  object 
 6   channel_entrace         812884 non-null  object 
 7   activity_status         949305 non-null  float64
 8   household_gross_income  709358 non-null  float64
 9   saving_account          956645 non-null  int64  
 10  guarantees              956645 non-null  int64  
 11  junior_account          956645 non-null  int64  
 12  loans                   956645 non-null  int64  
 13  credit_card             956645 non-null  int64  
 14  pension                

In [8]:
# Transform Gender
train_df['gender'] = np.random.choice(
    [0, 1],
    size = len(train_df),
    p = [0.6, 0.4]
)


# Transform Age
train_df['age'] = train_df['age'].apply(lambda x: str.strip(x))
train_df = train_df[train_df['age'] != 'NA']
train_df['age'] = train_df['age'].astype('int')
train_df = train_df[(train_df['age'] < 100) & (train_df['age'] > 18)]

# Transform first join date
train_df['first_join_date'] = pd.to_datetime(train_df['first_join_date'])
train_df['first_join_date'] = train_df['first_join_date'] + pd.DateOffset(years=5)

# Transform residence_index
train_df['residence_index'] = train_df['residence_index'].apply(lambda x: 'Y' if x == 'S' else 'N')

# Transform channel entrance
train_df['channel_entrace'] = train_df['channel_entrace'].fillna('KDB')

# Transform pension
train_df.loc[train_df['pension'].isna(), 'pension'] = np.random.randint(0, 2, size=train_df['pension'].isna().sum())
train_df['pension'] = train_df['pension'].astype('int')

In [9]:
# Maintain reproducibility of np.random
np.random.seed(42)

# Generate annual household gross income
mean_lognorm = np.log(34200).round(2)
sigma_lognorm = np.sqrt(2*(np.log(47000) - mean_lognorm)).round(2)
household_incomes = np.random.lognormal(mean=mean_lognorm, sigma=sigma_lognorm, size=len(train_df))
train_df['household_gross_income'] = household_incomes.round(2)

# Generate personal income
proportions = np.random.uniform(0.5, 1.0, size = len(train_df))
personal_incomes = proportions * household_incomes
train_df['personal_income'] = personal_incomes.round(2)

In [10]:
# Generate number of children
num_child = np.random.randint(0, 4, size=len(train_df))
train_df['number_of_children'] = num_child

# Generate employment status
employment_status = np.random.randint(0, 2, size = len(train_df))
train_df['employment_status'] = employment_status

In [11]:
# Regenerate activity status
train_df['activity_status'] = np.random.choice(
    [0, 1],
    size = len(train_df),
    p = [0.1, 0.9]
)

# Regenerate saving account values
train_df['saving_account'] = np.random.choice(
    [0, 1],
    size = len(train_df),
    p = [0.55, 0.45]
)

# Regenerate loans values
train_df['loans'] = np.random.choice(
    [0, 1],
    size = len(train_df),
    p = [0.4, 0.6]
)

# Regenerate credit card values
train_df['credit_card'] = np.random.choice(
    [0, 1],
    size = len(train_df),
    p = [0.5, 0.5]
)

# Generate current loan balance
loan_to_income_ratio = np.random.lognormal(mean = -0.4, sigma = 0.6, size = len(train_df))
loan_amount = train_df['personal_income'] * loan_to_income_ratio
train_df['current_loan_amount'] = train_df['loans'] * loan_amount
train_df['current_loan_amount'] = train_df['current_loan_amount'].round(2)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 939470 entries, 0 to 13647308
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   customer_id             939470 non-null  int64         
 1   residence_country       939470 non-null  object        
 2   gender                  939470 non-null  int64         
 3   age                     939470 non-null  int64         
 4   first_join_date         939470 non-null  datetime64[ns]
 5   residence_index         939470 non-null  object        
 6   channel_entrace         939470 non-null  object        
 7   activity_status         939470 non-null  int64         
 8   household_gross_income  939470 non-null  float64       
 9   saving_account          939470 non-null  int64         
 10  guarantees              939470 non-null  int64         
 11  junior_account          939470 non-null  int64         
 12  loans                   939470 no

In [13]:
# Hypothetical credit score generation
def generate_credit_score(row):
    base_score = np.random.normal(650, 100)

    # Income factor (higher income => higher score)
    temp = np.clip(row['household_gross_income'] / 10000, 0, 10)
    income_score = temp * 15
    base_score += income_score

    # Account history and activity status
    if row['activity_status'] == 1:
        account_age = (datetime.now() - row['first_join_date']).days / 365
        base_score += min(account_age * 10, 80)
    else:
        base_score -= 70
       
    # Product ownership 
    if row['saving_account'] == 1:
        base_score += 30
    if row['direct_debit'] == 1:
        base_score += 25
    if row['pension'] == 1:
        base_score += 40
        
    # Loan factor:
    if row['loans'] == 1:
        penalty = row['current_loan_amount'] * np.random.uniform(10, 25)
        base_score -= min(penalty, 120)
        
    # LTI - Loan to income factor:
    loan_amount = float(row['current_loan_amount'])
    personal_income = float(row['personal_income'])
    
    lti_ratio = loan_amount / personal_income
    if lti_ratio > 0.5:
        base_score -= min(int(lti_ratio * 200), 150)
    
    return max(0, min(int(base_score), 999))

In [14]:
train_df['credit_score'] = np.zeros(len(train_df))
train_df['credit_score'] = train_df.apply(lambda row: generate_credit_score(row), axis=1)
train_df.head(5)

,customer_id,residence_country,gender,age,first_join_date,residence_index,channel_entrace,activity_status,household_gross_income,saving_account,...,junior_account,loans,credit_card,pension,direct_debit,personal_income,number_of_children,employment_status,current_loan_amount,credit_score
0,1375586,ES,0,35,2020-01-12,Y,KHL,1,50887.44,1,...,0,0,0,0,0,31711.45,3,1,0.00,767
1,1050611,ES,1,23,2017-08-10,Y,KHE,1,30619.38,1,...,0,0,1,0,0,27281.84,0,0,0.00,717
2,1050612,ES,0,23,2017-08-10,Y,KHE,1,57420.17,0,...,0,1,0,0,0,28891.38,0,1,19164.84,514
3,1050613,ES,0,22,2017-08-10,Y,KHD,1,115661.59,0,...,0,1,0,0,0,87510.26,3,1,30865.72,761
4,1050614,ES,1,23,2017-08-10,Y,KHE,1,28358.36,0,...,0,1,1,0,0,24310.19,1,1,2491.76,714


In [15]:
train_df['first_join_date'].dt.year

0           2020
1           2017
2           2017
3           2017
4           2017
            ... 
13632377    2021
13641337    2018
13642462    2018
13645025    2021
13647308    2021
Name: first_join_date, Length: 939470, dtype: int32

In [16]:
# Create customer groups based on their first join date
def generate_segment(row):
    segment = ""
    years_since_join = 2020 - row['first_join_date'].year
    
    if years_since_join <= 1:
        segment = "0-1 year"
    elif 1 < years_since_join <= 4 :
        segment = "2-4 years"
    elif years_since_join >= 5:
        segment = "More than 5 years"
    else:
        raise Exception("Invalid date range. Date:{}".format(row['first_join_date'].year))
        
    return segment
        
train_df['customer_segment_model'] = train_df.apply(lambda row: generate_segment(row), axis=1)

In [17]:
train_df['customer_segment_model']

0            0-1 year
1           2-4 years
2           2-4 years
3           2-4 years
4           2-4 years
              ...    
13632377     0-1 year
13641337    2-4 years
13642462    2-4 years
13645025     0-1 year
13647308     0-1 year
Name: customer_segment_model, Length: 939470, dtype: object

In [18]:
# Export the final version of customers' profiles dataset
train_df.to_csv('project-dataset/customer.csv', index=False)